# Решение #1 - Фильтрация

### Импортирование библиотек и загрузка датасета

In [8]:
from src import *
import pandas as pd
df = pd.read_json('./../data/sample.json')
df['text'] = df['text'].apply(load_dataset.preprocess_text)

### Решение основано на поэтапном отборе групп

In [9]:
pd.options.mode.chained_assignment = None # Убираем лишние предупреждения
df['used'] = [False] * len(df)

def filter(groups,function,threshold,check_pronoun=True):
    for i in range(len(df)):
        if df['used'][i]:
            continue
        groups.append([])
        groups[-1].append(df['text'][i])
        df['used'][i] = True
        
        for j in range(i+1,len(df)):
            if df['used'][j]:
                continue
            if check_pronoun:
                if(function(df['text'][i],df['text'][j],threshold) and pronouns.check_pronoun_correspondence(df['text'][i],df['text'][j])):
                    groups[-1].append(df['text'][j])
                    df['used'][j] = True
            else:
                if(function(df['text'][i],df['text'][j],threshold)):
                    groups[-1].append(df['text'][j])
                    df['used'][j] = True
        if len(groups[-1]) == 1:
            df['used'][i] = False
            groups = groups[:-1]
    return groups

groups = []

groups = filter(groups,karlovskiy_distance.is_rewrite_karlovskiy_distance,0.89,check_pronoun=True)
groups = filter(groups,word_set.is_rewrite_word_set,threshold=0.99,check_pronoun=False)
groups = filter(groups,hamming_distance_custom.is_rewrite_hamming_distance_custom,threshold=0.9,check_pronoun=False)
groups = filter(groups,cosine_tfidf.is_rewrite_cosine_tfidf,0.7,check_pronoun=True)
groups = filter(groups,jaro_winkler.is_rewrite_jaro_winkler,0.9,check_pronoun=True)

groups = groups + [[str] for str in list(df[df['used']==False]['text'])]

df = df.drop('used',axis=1)

### Все найденные группы

In [10]:
groups

[['никто туда больше не ходит', 'никто больше туда не ходит'],
 ['у него с собой не было тогда денег', 'у него тогда не было с собой денег'],
 ['я больше не хочу с тобой играть', 'я не хочу с тобой больше играть'],
 ['что сделал том с деньгами', 'что сделел том с деньгами'],
 ['том меня сейчас хочет видеть', 'том хочет меня сейчас видеть'],
 ['я его больше не увижу', 'я больше его не увижу'],
 ['том и мэри объявили сегодня о своей пбмолвке',
  'том и мэри объявили сегодня о своей помолвке'],
 ['я не могу больше ждать', 'я больше не могу ждать'],
 ['мост очень длинный и высокий', 'мост очень длинный и очень высокий'],
 ['пусть свиньи это едят', 'пусть это свиньи едят'],
 ['ты хотел мне рассказать о свободе', 'ты хотел рассказать мне о свободе'],
 ['что пел джон на сцене', 'что джон пел на сцене'],
 ['я написал влера письмо', 'я написал вчера письмо'],
 ['мы не были готовы', 'мы были не готовы'],
 ['они их только что нашли', 'они только что их нашли'],
 ['том не знает ничего о семье мэри

### Строки, которые имеют более 1 рерайта

In [11]:
[group for group in groups if len(group) > 2]

[['том не хочет никого видеть',
  'том не хочет сегодня никого видеть',
  'том сегодня не хочет никого видеть']]

### Строки, которые имеют ровно 1 рерайт

In [12]:
[group for group in groups if len(group) == 2]

[['никто туда больше не ходит', 'никто больше туда не ходит'],
 ['у него с собой не было тогда денег', 'у него тогда не было с собой денег'],
 ['я больше не хочу с тобой играть', 'я не хочу с тобой больше играть'],
 ['что сделал том с деньгами', 'что сделел том с деньгами'],
 ['том меня сейчас хочет видеть', 'том хочет меня сейчас видеть'],
 ['я его больше не увижу', 'я больше его не увижу'],
 ['том и мэри объявили сегодня о своей пбмолвке',
  'том и мэри объявили сегодня о своей помолвке'],
 ['я не могу больше ждать', 'я больше не могу ждать'],
 ['мост очень длинный и высокий', 'мост очень длинный и очень высокий'],
 ['пусть свиньи это едят', 'пусть это свиньи едят'],
 ['ты хотел мне рассказать о свободе', 'ты хотел рассказать мне о свободе'],
 ['что пел джон на сцене', 'что джон пел на сцене'],
 ['я написал влера письмо', 'я написал вчера письмо'],
 ['мы не были готовы', 'мы были не готовы'],
 ['они их только что нашли', 'они только что их нашли'],
 ['том не знает ничего о семье мэри

### Строки, которые не имеют рерайта

In [13]:
[group for group in groups if len(group) == 1]

[['ты нашел их или нет'],
 ['почему она так со мной поступает'],
 ['почему они с нами так поступают'],
 ['он всю ночь стонал от сильной боли'],
 ['тому было тогда всего тринадцать лет'],
 ['он даже меня не замечает'],
 ['тебе это все нравится'],
 ['почему она так с ней поступает'],
 ['я хотел бы учиться в бостоне'],
 ['том этим сейчас занимается'],
 ['том был просто не готов'],
 ['тому это тоже не нравится'],
 ['сколько сейчас времени в париже'],
 ['тому было больше не к кому обратиться'],
 ['он сказал что вчера был дома'],
 ['они никогда меня не слушают'],
 ['я поймал сегодня три рыбы'],
 ['я хочу что-нибудь сделать для тома'],
 ['я сделал все правильно'],
 ['я тоже завтра пойду в университет'],
 ['мне никогда не нужно было столько денег'],
 ['том не может жить без мэри'],
 ['почему они так с ним поступают'],
 ['что могло с ними случиться'],
 ['мы будем делать все по-своему'],
 ['где был том весь день'],
 ['я здесь часто ем'],
 ['я все ей рассказываю'],
 ['том больше мне не доверяет']

### Сохранение результата

In [14]:
save_groups.save_groups(groups,"../output/solution1-result.json")